## Preliminary Project 1 Notebook
Note: It needs more data formatting for final report

#### Import Modules

In [890]:
import pandas as pd
import numpy as np
import seaborn as sns

#### Loading Data

In [1026]:
sat17_df = pd.read_csv('data/sat_2017.csv')
sat18_df = pd.read_csv('data/sat_2018.csv')
sat19_df = pd.read_csv('data/sat_2019.csv')
act17_df = pd.read_csv('data/act_2017.csv')
act18_df = pd.read_csv('data/act_2018.csv')
act19_df = pd.read_csv('data/act_2019.csv')
state_region_df = pd.read_csv('data/us_state_region.csv')

sat17_df['Year'] = 2017
sat17_df['Type'] = 'SAT'
sat18_df['Year'] = 2018
sat18_df['Type'] = 'SAT'
sat19_df['Year'] = 2019
sat19_df['Type'] = 'SAT'

act17_df['Year'] = 2017
act17_df['Type'] = 'ACT'
act18_df['Year'] = 2018
act18_df['Type'] = 'ACT'
act19_df['Year'] = 2019
act19_df['Type'] = 'ACT'


exam_df = pd.concat([sat17_df, sat18_df, sat19_df, act17_df, act18_df, act19_df], ignore_index=True)

### Data Exploration

In [ ]:
# review sample data
for year, type in [[2017, 'SAT'], [2018, 'SAT'], [2019, 'SAT'], [2017, 'ACT'], [2018, 'ACT'], [2019, 'ACT']]:
    print(f"Year:{year}, Type:{type} Shape:{exam_df[(exam_df['Year']==year) & (exam_df['Type']==type)].shape}")
    print("* Review Sample Data")
    print(exam_df[(exam_df['Year']==year) & (exam_df['Type']==type)].head(5))
    print("\n* Review NULL Data")
    print(exam_df[(exam_df['Year']==year) & (exam_df['Type']==type)].isna().sum())
    print("\n* Review data type")
    print(exam_df[(exam_df['Year']==year) & (exam_df['Type']==type)].dtypes)
    print('-' * 80, '\n')

### Cleaning Data
SAT 19 has difference column name, EBRW vs. Evidence-Based Reading and Writing

In [1121]:
# SAT 2019 has difference column name, EBRW vs. Evidence-Based Reading and Writing
sat19_df.rename(columns={'EBRW': 'Evidence-Based Reading and Writing', 'Participation Rate': 'Participation'}, inplace=True)

# re-combine rename dataframe
exam_df = pd.concat([sat17_df, sat18_df, sat19_df, act17_df, act18_df, act19_df], ignore_index=True)
exam_df['Participation'] = pd.to_numeric(exam_df['Participation'].str.replace(r'—|%','', regex=True), errors='coerce')

# recheck null Participation
exam_df[exam_df['Participation'].isnull()]
exam_df['Participation'].isnull().sum()
# since it only 2 rows, we decide to drop these rows
exam_df = exam_df[~exam_df['Participation'].isna()]
# change Participation data type to int64
exam_df.loc[:, 'Participation'] = exam_df['Participation'].astype('int64')
#exam_df['Participation'] = exam_df['Participation'].astype('int64')

# mapping state with region
exam_df = exam_df.merge(state_region_df)

#### Formatting Data

In [1141]:
# select 
exam_df.groupby(['Year','Type','Region'])['Participation'].describe()

count       mean        std   min    25%    50%     75%  \
Year Type Region                                                               
2017 ACT  Midwest     12.0  77.833333  24.349102  29.0  71.50   82.0   98.50   
          Northeast    9.0  24.888889   8.237786   8.0  21.00   29.0   31.00   
          South       17.0  73.470588  31.965054  18.0  45.00  100.0  100.00   
          West        13.0  70.846154  29.042630  29.0  40.00   66.0  100.00   
     SAT  Midwest     12.0  17.250000  31.177279   2.0   3.00    3.0    9.75   
          Northeast    9.0  77.777778  15.114379  60.0  67.00   71.0   95.00   
          South       17.0  40.176471  36.435277   2.0   5.00   49.0   65.00   
          West        13.0  33.846154  27.257556   3.0  11.00   30.0   53.00   
2018 ACT  Midwest     12.0  75.833333  29.151121  22.0  61.75   87.5  100.00   
          Northeast   10.0  19.800000   8.337332   7.0  15.25   22.0   25.75   
          South       16.0  75.062500  31.718488  17.0  51.00  100.0  100.00   
          West        13.0  62.615385  31.821215  24.0  33.00   66.0  100.00   
     SAT  Midwest     12.0  25.500000  38.587563   2.0   3.00    4.0   29.25   
          Northeast    9.0  85.222222  13.330208  64.0  79.00   82.0   97.00   
          South       17.0  41.117647  34.667857   3.0   6.00   52.0   68.00   
          West        13.0  43.153846  33.155809   3.0  16.00   43.0   60.00   
2019 ACT  Midwest     12.0  72.416667  29.583344  19.0  58.25   78.5   97.00   
          Northeast    9.0  17.666667   6.020797   6.0  14.00   20.0   22.00   
          South       17.0  68.411765  33.706562  13.0  39.00   78.0  100.00   
          West        13.0  61.615385  32.076831  23.0  31.00   63.0  100.00   
     SAT  Midwest     12.0  25.916667  39.016217   2.0   3.00    4.0   30.75   
          Northeast    9.0  85.777778  13.169831  66.0  79.00   82.0   99.00   
          South       17.0  50.294118  39.164341   3.0   7.00   68.0   82.00   
          West        13.0  43.384615  33.402940   3.0  18.00   41.0   63.00   

                       max  
Year Type Region            
2017 ACT  Midwest    100.0  
          Northeast   34.0  
          South      100.0  
          West       100.0  
     SAT  Midwest    100.0  
          Northeast  100.0  
          South      100.0  
          West        93.0  
2018 ACT  Midwest    100.0  
          Northeast   31.0  
          South      100.0  
          West       100.0  
     SAT  Midwest    100.0  
          Northeast  100.0  
          South      100.0  
          West       100.0  
2019 ACT  Midwest    100.0  
          Northeast   25.0  
          South      100.0  
          West       100.0  
     SAT  Midwest    100.0  
          Northeast  100.0  
          South      100.0  
          West       100.0

#### Visualization

In [1149]:
# visualization code